In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.plotting import scatter_matrix
from pandas_profiling import ProfileReport

In [31]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import make_pipeline

In [4]:
ins_df = pd.read_csv('insurance.csv')
ins_df

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


In [5]:
ins_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [7]:
ins_df.nunique()

age           47
sex            2
bmi          548
children       6
smoker         2
region         4
charges     1337
dtype: int64

In [13]:
ins_df_onehot = pd.get_dummies(ins_df, columns=['sex', 'smoker', 'region']).drop('charges', axis=1)
ins_df_target = ins_df['charges']
ins_df_onehot

,age,bmi,children,sex_female,sex_male,smoker_no,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest
0,19,27.900,0,1,0,0,1,0,0,0,1
1,18,33.770,1,0,1,1,0,0,0,1,0
2,28,33.000,3,0,1,1,0,0,0,1,0
3,33,22.705,0,0,1,1,0,0,1,0,0
4,32,28.880,0,0,1,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
1333,50,30.970,3,0,1,1,0,0,1,0,0
1334,18,31.920,0,1,0,1,0,1,0,0,0
1335,18,36.850,0,1,0,1,0,0,0,1,0
1336,21,25.800,0,1,0,1,0,0,0,0,1


In [16]:
X_train, X_test, y_train, y_test = train_test_split(ins_df_onehot.values, ins_df_target.values, random_state=0)

In [17]:
pipe = make_pipeline(MinMaxScaler(), LinearRegression()).fit(X_train, y_train)
pipe.score(X_test, y_test)

0.7958786376014413

In [19]:
y_pred = pipe.predict(X_test)

In [20]:
pd.DataFrame(zip(y_test, y_pred), columns=['actual', 'predict'])

,actual,predict
0,9724.53000,11121.101409
1,8547.69130,9369.083452
2,45702.02235,38349.258807
3,12950.07120,16331.935006
4,9644.25250,7041.227706
...,...,...
330,13224.05705,15034.106647
331,4433.91590,7121.723799
332,9377.90470,10602.930893
333,3597.59600,7050.864796


In [23]:
pipe.named_steps['linearregression'].coef_

array([ 1.15069979e+04,  1.23886402e+04,  2.21106880e+03, -6.63153624e+00,
        6.63153624e+00, -1.18283103e+04,  1.18283103e+04,  5.35344576e+02,
        3.42211643e+02, -5.44457080e+02, -3.33099139e+02])

In [30]:
pipe2 = make_pipeline(MinMaxScaler(), Lasso(alpha=0.1)).fit(X_train, y_train)
pipe2.score(X_test, y_test)

0.7958757989117826